In [ ]:
using Revise

In [ ]:
using MeshCat
using RigidBodyDynamics
using MeshCatMechanisms

In [ ]:
vis = Visualizer()
open(vis)  # open the visualizer in a separate tab/window
IJuliaCell(vis)

In [ ]:
robot = parse_urdf(Float64, "test/urdf/Acrobot.urdf")
delete!(vis)
mvis = MechanismVisualizer(robot, "test/urdf/Acrobot.urdf", vis)
state = MechanismState(robot)
set_configuration!(state, [1.0, -0.5])
setstate!(mvis, state)

In [ ]:
t, q, v = simulate(state, 5.0);

In [ ]:
for i in 1:10:length(t)
    set_configuration!(state, q[i])
    setstate!(mvis, state)
    if i < length(t)
        sleep(t[i + 1] - t[i])
    end
end

In [ ]:
using ValkyrieRobot

val = Valkyrie();

delete!(vis)
mvis = MechanismVisualizer(val.mechanism, ValkyrieRobot.urdfpath(), vis, package_path=[dirname(dirname(ValkyrieRobot.urdfpath()))])

state = MechanismState(val.mechanism)
setstate!(mvis, state)

In [ ]:
using ValkyrieRobot.BipedControlUtil: Side, flipsign_if_right

function initialize(state::MechanismState, val::Valkyrie, vis::MechanismVisualizer)
    zero!(state)
    mechanism = val.mechanism
    for side in instances(Side)
        set_configuration!(state, findjoint(mechanism, "$(side)KneePitch"), [1.205])
        set_configuration!(state, findjoint(mechanism, "$(side)HipPitch"), [-0.49])
        set_configuration!(state, findjoint(mechanism, "$(side)AnklePitch"), [-0.71])
        set_configuration!(state, findjoint(mechanism, "$(side)ShoulderPitch"), [0.300196631343025])
        set_configuration!(state, findjoint(mechanism, "$(side)ShoulderRoll"), [flipsign_if_right(-1.25, side)])
        set_configuration!(state, findjoint(mechanism, "$(side)ElbowPitch"), [flipsign_if_right(-0.785398163397448, side)])
        set_configuration!(state, findjoint(mechanism, "$(side)ForearmYaw"), [1.571])
    end
    set_configuration!(state, val.basejoint, [1; 0; 0; 0; 0; 0; 1.025])
    setstate!(vis, state)
    nothing
end

initialize(state, val, mvis)

In [ ]:
robot = parse_urdf(Float64, "test/urdf/Acrobot_fixed.urdf")
RigidBodyDynamics.remove_fixed_tree_joints!(robot)
delete!(vis)
mvis = MechanismVisualizer(robot, "test/urdf/Acrobot_fixed.urdf", vis)

state = MechanismState(robot)
set_configuration!(state, [0.5])
setstate!(mvis, state)

In [ ]:
for theta in linspace(-pi, pi, 1000)
    set_configuration!(state, [theta])
    setstate!(mvis, state)
    sleep(0.001)
end